In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%cd '../HOTS'

/Users/laurentperrinet/quantic/science/HomeHots/HOTS_clone_laurent/HOTS


In [2]:
import os
import pickle
import datetime
from mix_Network import *

dataset = 'nmnist'
path = f'../Records/{dataset}/models/'
timestr = '2021-02-16'

%mkdir -p ../Records
#%mkdir -p ../Records/EXP_03_NMNIST

sigma = None
pooling = False
homeinv = False
jitonic = [None,None] #[temporal, spatial]
jitter = False
tau = 5
nb_train = 5000
nb_test = 1000
nblay = 3
nbclust = 4
filt = 2

ds = 50
nb_train_b = nb_train//ds
nb_test_b = nb_test//ds

In [3]:
def netparam(name, filt, tau, nblay, nbclust, sigma, homeinv, jitter):
    if name=='hots':
        homeo = False
        homeotest = False
        krnlinit = 'first'
        hotshom = network(krnlinit=krnlinit, filt=filt, tau=tau, nblay=nblay, nbclust=nbclust, homeo=homeo, sigma=sigma, homeinv=homeinv, jitter=jitter)
        hotshom = hotshom.learning1by1()
    elif name=='homhots':
        homeo = True
        homeotest = True
        krnlinit = 'rdn'
        hotshom = network(krnlinit=krnlinit, filt=filt, tau=tau, nblay=nblay, nbclust=nbclust, homeo=homeo, sigma=sigma, homeinv=homeinv, jitter=jitter)
        hotshom = hotshom.learningall()
    elif name=='onlyonline':
        homeo = False
        homeotest = False
        krnlinit = 'rdn'
        hotshom = network(krnlinit=krnlinit, filt=filt, tau=tau, nblay=nblay, nbclust=nbclust, homeo=homeo, sigma=sigma, homeinv=homeinv, jitter=jitter)
        hotshom = hotshom.learningall()
    return hotshom, homeotest

In [4]:
name = 'homhots'
hotshom, homeotest = netparam(name, filt, tau, nblay, nbclust, sigma, homeinv, jitter)
histo_train = hotshom.running(homeotest = homeotest, nb_digit = nb_train, LR=False)
histo_test = hotshom.running(homeotest = homeotest, nb_digit = nb_test, LR=False)
JS_score = histoscore(trainhistomap,testhistomap, verbose = True)

2021-02-16_lagorce_rdn_None_True_[0.25, 1]_[4, 8, 16]_[5000.0, 50000.0, 500000.0]_[2, 4, 8]_False
../Records/nmnist/train/2021-02-16_lagorce_rdn_None_True_[0.25, 1]_[4, 8, 16]_[5000.0, 50000.0, 500000.0]_[2, 4, 8]_False_5000_[None, None]_homeo_histo.pkl
../Records/nmnist/train/2021-02-16_lagorce_rdn_None_True_[0.25, 1]_[4, 8, 16]_[5000.0, 50000.0, 500000.0]_[2, 4, 8]_False_1000_[None, None]_homeo_histo.pkl
no jitter


0it [00:00, ?it/s]

Extracting ../Data/nmnist_train.zip to ../Data/


  0%|          | 0/1000 [00:00<?, ?it/s]

NameError: name 'trainhistomap' is not defined

# Performing logistic regression on raw input spikes

# Performing logistic regression on the network output
## With homeostasis